In [1]:
# required libraries
import numpy as np
import pandas as pd
import os
import pydicom
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image.dataframe_iterator import DataFrameIterator
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping

In [2]:
root_dir = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
df = pd.read_csv(root_dir+'train_labels.csv')
df_test = pd.read_csv(root_dir+'sample_submission.csv')

In [3]:
# full paths for each id for different types of sequences
def full_ids(data):
    zeros = 5 - len(str(data))
    if zeros > 0:
        prefix = ''.join(['0' for i in range(zeros)])
    
    return prefix+str(data)
        

df['BraTS21ID_full'] = df['BraTS21ID'].apply(full_ids)
df_test['BraTS21ID_full'] = df_test['BraTS21ID'].apply(full_ids)

# adding all the paths to the train df for easy access
df['flair'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/FLAIR/')
df['t1w'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/T1w/')
df['t1wce'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/T1wCE/')
df['t2w'] = df['BraTS21ID_full'].apply(lambda file_id : root_dir+'train/'+file_id+'/T2w/')


# adding all the paths to the test df for easy access
df_test['flair'] = df_test['BraTS21ID_full'].apply(lambda file_id : root_dir+'test/'+file_id+'/FLAIR/')
df_test['t1w'] = df_test['BraTS21ID_full'].apply(lambda file_id : root_dir+'test/'+file_id+'/T1w/')
df_test['t1wce'] = df_test['BraTS21ID_full'].apply(lambda file_id : root_dir+'test/'+file_id+'/T1wCE/')
df_test['t2w'] = df_test['BraTS21ID_full'].apply(lambda file_id : root_dir+'test/'+file_id+'/T2w/')

In [4]:
df.head()

,BraTS21ID,MGMT_value,BraTS21ID_full,flair,t1w,t1wce,t2w
0,0,1,00000,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
1,2,1,00002,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
2,3,0,00003,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
3,5,1,00005,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
4,6,1,00006,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...


In [5]:
df_test.head()

,BraTS21ID,MGMT_value,BraTS21ID_full,flair,t1w,t1wce,t2w
0,1,0.5,00001,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
1,13,0.5,00013,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
2,15,0.5,00015,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
3,27,0.5,00027,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...
4,37,0.5,00037,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...,../input/rsna-miccai-brain-tumor-radiogenomic-...


# Load images

In [6]:
def get_train_val_dataframe(mri_type):
    all_img_files = []
    all_img_labels = []
    all_img_patient_ids = []
    for row in df.iterrows():
        # 00109 (FLAIR images are blank), 00123 (T1w images are blank), 00709 (FLAIR images are blank)
        if row[1]['BraTS21ID_full'] == '00109' and mri_type == 'flair':
            continue
        if row[1]['BraTS21ID_full'] == '00123' and mri_type == 't1w':
            continue
        if row[1]['BraTS21ID_full'] == '00709' and mri_type == 'flair':
            continue
        img_dir = row[1][mri_type]
        img_files = os.listdir(img_dir)
        img_nums = sorted([int(ele.replace('Image-', '').replace('.dcm', '')) for ele in img_files])
        mid_point = int(len(img_nums)/2)
        start_point = mid_point - max(int(mid_point*0.1), 1)
        end_point = mid_point + max(int(mid_point*0.1), 1)
        img_names = [f'Image-{img_nums[i]}.dcm' for i in range(start_point, end_point+1)]
        img_paths = [img_dir+ele for ele in img_names]
        img_labels = [row[1]['MGMT_value']]*len(img_paths)
        img_patient_ids = [row[1]['BraTS21ID']]*len(img_paths)
        all_img_files.extend(img_paths)
        all_img_labels.extend(img_labels)
        all_img_patient_ids.extend(img_patient_ids)

    train_val_df = pd.DataFrame({'patient_ids': all_img_patient_ids,
                  'labels': all_img_labels,
                  'file_paths': all_img_files})

    train_val_df['labels'] = train_val_df['labels'].map({1: '1', 0: '0'})
    
    return train_val_df
    
def get_test_dataframe(mri_type):
    all_test_img_files = []
    all_test_img_labels = []
    all_test_img_patient_ids = []
    for row in df_test.iterrows():
        img_dir = row[1][mri_type]
        img_files = os.listdir(img_dir)
        img_nums = sorted([int(ele.replace('Image-', '').replace('.dcm', '')) for ele in img_files])
        mid_point = int(len(img_nums)/2)
        start_point = mid_point - max(int(mid_point*0.1), 1)
        end_point = mid_point + max(int(mid_point*0.1), 1)
        img_names = [f'Image-{img_nums[i]}.dcm' for i in range(start_point, end_point+1)]
        img_paths = [img_dir+ele for ele in img_names]
        img_labels = [row[1]['MGMT_value']]*len(img_paths)
        img_patient_ids = [row[1]['BraTS21ID']]*len(img_paths)
        all_test_img_files.extend(img_paths)
        all_test_img_labels.extend(img_labels)
        all_test_img_patient_ids.extend(img_patient_ids)

    test_df = pd.DataFrame({'patient_ids': all_test_img_patient_ids,
                  'labels': all_test_img_labels,
                  'file_paths': all_test_img_files})
    
    test_df['labels'] = ['1']*(len(test_df)-1) + ['0'] # workaround for testing data gen
    
    return test_df

In [7]:
class DCMDataFrameIterator(DataFrameIterator):
    def __init__(self, *arg, **kwargs):
        self.white_list_formats = ('dcm')
        super(DCMDataFrameIterator, self).__init__(*arg, **kwargs)
        self.dataframe = kwargs['dataframe']
        self.x = self.dataframe[kwargs['x_col']]
        self.y = self.dataframe[kwargs['y_col']]
        self.color_mode = kwargs['color_mode']
        self.target_size = kwargs['target_size']

    def _get_batches_of_transformed_samples(self, indices_array):
        # get batch of images
        batch_x = np.array([self.read_dcm_as_array(dcm_path, self.target_size, color_mode=self.color_mode)
                            for dcm_path in self.x.iloc[indices_array]])

        batch_y = np.array(self.y.iloc[indices_array].astype(np.uint8))  # astype because y was passed as str

        # transform images
        if self.image_data_generator is not None:
            for i, (x, y) in enumerate(zip(batch_x, batch_y)):
                transform_params = self.image_data_generator.get_random_transform(x.shape)
                batch_x[i] = self.image_data_generator.apply_transform(x, transform_params)

        return batch_x, batch_y

    @staticmethod
    def read_dcm_as_array(dcm_path, target_size=(224, 224), color_mode='rgb'):
        image_array = pydicom.dcmread(dcm_path).pixel_array
        pixels = image_array - np.min(image_array)
        pixels = pixels / np.max(pixels)
        image_manual_norm = (pixels * 255).astype(np.uint8)
        image_array = cv2.resize(image_manual_norm, target_size, interpolation=cv2.INTER_NEAREST)
        if color_mode == 'rgb':
            image_array = np.dstack((image_array, np.zeros_like(image_array), np.zeros_like(image_array)))
        return image_array

In [8]:
# constants
SEED = 369
BATCH_SIZE = 512
CLASS_MODE = 'binary'
COLOR_MODE = 'rgb'
TARGET_SIZE = (224, 224)

In [9]:
def get_data_generators(train_df, test_df):
    train_augmentation_parameters = dict(
    #     rescale=1.0/255.0,
        zoom_range=0.2,
        fill_mode='nearest',
        height_shift_range= 0.1,
        width_shift_range=0.1,
        brightness_range = [0.8, 1.2]
    )

    test_augmentation_parameters = dict(
    #     rescale=1.0/255.0
    )

    train_consts = {
        'seed': SEED,
        'batch_size': BATCH_SIZE,
        'class_mode': CLASS_MODE,
        'color_mode': COLOR_MODE,
        'target_size': TARGET_SIZE,  
    }

    test_consts = {
        'batch_size': BATCH_SIZE,
        'class_mode': CLASS_MODE,
        'color_mode': COLOR_MODE,
        'target_size': TARGET_SIZE,
        'shuffle': False
    }

    train_augmenter = ImageDataGenerator(**train_augmentation_parameters)
    test_augmenter = ImageDataGenerator(**test_augmentation_parameters)

    train_generator = DCMDataFrameIterator(dataframe=train_df,
                                 x_col='file_paths',
                                 y_col='labels',
                                 image_data_generator=train_augmenter,
                                 **train_consts)

    test_generator = DCMDataFrameIterator(dataframe=test_df,
                                 x_col='file_paths',
                                 y_col='labels',
                                 image_data_generator=test_augmenter,
                                 **test_consts)
    
    return train_generator, test_generator

In [10]:
train_df_flair = get_train_val_dataframe('flair')
train_df_t1w = get_train_val_dataframe('t1w')
train_df_t1wce = get_train_val_dataframe('t1wce')
train_df_t2w = get_train_val_dataframe('t2w')

In [11]:
train_df_flair.head()

,patient_ids,labels,file_paths
0,0,1,../input/rsna-miccai-brain-tumor-radiogenomic-...
1,0,1,../input/rsna-miccai-brain-tumor-radiogenomic-...
2,0,1,../input/rsna-miccai-brain-tumor-radiogenomic-...
3,0,1,../input/rsna-miccai-brain-tumor-radiogenomic-...
4,0,1,../input/rsna-miccai-brain-tumor-radiogenomic-...


In [12]:
test_df_flair = get_test_dataframe('flair')
test_df_t1w = get_test_dataframe('t1w')
test_df_t1wce = get_test_dataframe('t1wce')
test_df_t2w = get_test_dataframe('t2w')

In [13]:
train_df_flair.shape, train_df_t1w.shape, train_df_t1wce.shape, train_df_t2w.shape

((7639, 3), (7736, 3), (9699, 3), (10155, 3))

In [14]:
train_flair, test_flair = get_data_generators(train_df_flair, test_df_flair)
train_t1w, test_t1w = get_data_generators(train_df_t1w, test_df_t1w)
train_t1wce, test_t1wce = get_data_generators(train_df_t1wce, test_df_t1wce)
train_t2w, test_t2w = get_data_generators(train_df_t2w, test_df_t2w)

Found 7639 validated image filenames belonging to 2 classes.
Found 1119 validated image filenames belonging to 2 classes.
Found 7736 validated image filenames belonging to 2 classes.
Found 1187 validated image filenames belonging to 2 classes.
Found 9699 validated image filenames belonging to 2 classes.
Found 1441 validated image filenames belonging to 2 classes.
Found 10155 validated image filenames belonging to 2 classes.
Found 1463 validated image filenames belonging to 2 classes.


# Models

In [15]:
def model_EfficientNetB0(path, lr = 0.001, dr_rate = 0.15):
    model = EfficientNetB0(include_top=False, weights=path)
    model.trainable = False

    # Rebuild top
    x = GlobalAveragePooling2D()(model.output)
    x = BatchNormalization()(x)
    x = Dropout(dr_rate)(x)
    dense_1 = Dense(32, activation="relu")(x)
    outputs = Dense(1, activation="sigmoid")(dense_1)

    # Compile
    model = Model(model.inputs, outputs, name="EfficientNet")
    optimizer = Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["binary_accuracy"]
    )
    return model
    
    

In [16]:
def model_EfficientNetB1(path, lr = 0.001, dr_rate = 0.15):
    model = EfficientNetB1(include_top=False, weights=path)
    model.trainable = False

    # Rebuild top
    x = GlobalAveragePooling2D()(model.output)
    x = BatchNormalization()(x)
    x = Dropout(dr_rate)(x)
    dense_1 = Dense(32, activation="relu")(x)
    outputs = Dense(1, activation="sigmoid")(dense_1)

    # Compile
    model = Model(model.inputs, outputs, name="EfficientNet")
    optimizer = Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model

In [17]:
def model_EfficientNetB2(path, lr = 0.001, dr_rate = 0.15):
    model = EfficientNetB2(include_top=False, weights=path)
    model.trainable = False

    # Rebuild top
    x = GlobalAveragePooling2D()(model.output)
    x = BatchNormalization()(x)
    x = Dropout(dr_rate)(x)
    dense_1 = Dense(32, activation="relu")(x)
    outputs = Dense(1, activation="sigmoid")(dense_1)

    # Compile
    model = Model(model.inputs, outputs, name="EfficientNet")
    optimizer = Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model

In [18]:
def model_EfficientNetB3(path, lr = 0.001, dr_rate = 0.15):
    model = EfficientNetB1(include_top=False, weights=path)
    model.trainable = False

    # Rebuild top
    x = GlobalAveragePooling2D()(model.output)
    x = BatchNormalization()(x)
    x = Dropout(dr_rate)(x)
    dense_1 = Dense(32, activation="relu")(x)
    outputs = Dense(1, activation="sigmoid")(dense_1)

    # Compile
    model = Model(model.inputs, outputs, name="EfficientNet")
    optimizer = Adam(learning_rate=lr)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model
        

In [19]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# EfficientNet-B0

In [20]:
efficientB0_flaire = model_EfficientNetB0('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b0_tf24_imagenet_1000_notop.h5')
history_flair_1 = efficientB0_flaire.fit_generator(
        generator=train_flair,
        steps_per_epoch=len(train_flair),
        epochs=10,
        validation_data=test_flair,
        validation_steps=len(test_flair),
        callbacks=[callback],
        workers=2)

2021-10-20 08:03:33.609718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-20 08:03:33.707436: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-20 08:03:33.708512: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-20 08:03:33.710336: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/10


2021-10-20 08:04:10.477939: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


15/15 [==============================] - 142s 8s/step - loss: 0.7206 - binary_accuracy: 0.5816 - val_loss: 0.5598 - val_binary_accuracy: 0.9312
Epoch 2/10
15/15 [==============================] - 113s 6s/step - loss: 0.6471 - binary_accuracy: 0.6312 - val_loss: 0.6280 - val_binary_accuracy: 0.7605
Epoch 3/10
15/15 [==============================] - 101s 6s/step - loss: 0.6146 - binary_accuracy: 0.6607 - val_loss: 0.5761 - val_binary_accuracy: 0.8686
Epoch 4/10
15/15 [==============================] - 102s 6s/step - loss: 0.5919 - binary_accuracy: 0.6769 - val_loss: 0.6193 - val_binary_accuracy: 0.7373


In [21]:
efficientB0_t1w = model_EfficientNetB0('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b0_tf24_imagenet_1000_notop.h5')
history_t1w_1 = efficientB0_t1w.fit_generator(
        generator=train_t1w,
        steps_per_epoch=len(train_t1w),
        epochs=10,
        validation_data=test_t1w,
        validation_steps=len(test_t1w),
        callbacks=[callback],
        workers=2)

Epoch 1/10
16/16 [==============================] - 172s 9s/step - loss: 0.7557 - binary_accuracy: 0.5486 - val_loss: 0.6521 - val_binary_accuracy: 0.6714
Epoch 2/10
16/16 [==============================] - 114s 6s/step - loss: 0.6720 - binary_accuracy: 0.6038 - val_loss: 0.5954 - val_binary_accuracy: 0.8526
Epoch 3/10
16/16 [==============================] - 110s 6s/step - loss: 0.6388 - binary_accuracy: 0.6368 - val_loss: 0.6612 - val_binary_accuracy: 0.6049
Epoch 4/10
16/16 [==============================] - 100s 6s/step - loss: 0.6237 - binary_accuracy: 0.6511 - val_loss: 0.6434 - val_binary_accuracy: 0.6512
Epoch 5/10
16/16 [==============================] - 100s 6s/step - loss: 0.6061 - binary_accuracy: 0.6665 - val_loss: 0.6203 - val_binary_accuracy: 0.7211


In [22]:
efficientB0_t1wce = model_EfficientNetB0('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b0_tf24_imagenet_1000_notop.h5')
history_t1wce_1 = efficientB0_t1wce.fit_generator(
        generator=train_t1wce,
        steps_per_epoch=len(train_t1wce),
        epochs=10,
        validation_data=test_t1wce,
        validation_steps=len(test_t1wce),
        callbacks=[callback],
        workers=2)

Epoch 1/10
19/19 [==============================] - 204s 9s/step - loss: 0.7447 - binary_accuracy: 0.5632 - val_loss: 0.7543 - val_binary_accuracy: 0.3012
Epoch 2/10
19/19 [==============================] - 144s 7s/step - loss: 0.6629 - binary_accuracy: 0.6206 - val_loss: 0.6754 - val_binary_accuracy: 0.5684
Epoch 3/10
19/19 [==============================] - 127s 6s/step - loss: 0.6323 - binary_accuracy: 0.6445 - val_loss: 0.7641 - val_binary_accuracy: 0.3178
Epoch 4/10
19/19 [==============================] - 136s 6s/step - loss: 0.6056 - binary_accuracy: 0.6676 - val_loss: 0.7317 - val_binary_accuracy: 0.3865
Epoch 5/10
19/19 [==============================] - 136s 6s/step - loss: 0.5950 - binary_accuracy: 0.6777 - val_loss: 0.7462 - val_binary_accuracy: 0.3720


In [23]:
efficientB0_t2w = model_EfficientNetB0('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b0_tf24_imagenet_1000_notop.h5')
history_t2w_1 = efficientB0_t2w.fit_generator(
        generator=train_t2w,
        steps_per_epoch=len(train_t2w),
        epochs=10,
        validation_data=test_t2w,
        validation_steps=len(test_t2w),
        callbacks=[callback],
        workers=2)

Epoch 1/10


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


20/20 [==============================] - 223s 10s/step - loss: 0.7246 - binary_accuracy: 0.5806 - val_loss: 0.5450 - val_binary_accuracy: 0.9638
Epoch 2/10
20/20 [==============================] - 140s 7s/step - loss: 0.6306 - binary_accuracy: 0.6491 - val_loss: 0.5421 - val_binary_accuracy: 0.9392
Epoch 3/10
20/20 [==============================] - 146s 7s/step - loss: 0.5990 - binary_accuracy: 0.6759 - val_loss: 0.5466 - val_binary_accuracy: 0.9118
Epoch 4/10
20/20 [==============================] - 136s 6s/step - loss: 0.5729 - binary_accuracy: 0.6913 - val_loss: 0.5180 - val_binary_accuracy: 0.9398
Epoch 5/10
20/20 [==============================] - 145s 7s/step - loss: 0.5521 - binary_accuracy: 0.7129 - val_loss: 0.5965 - val_binary_accuracy: 0.7915
Epoch 6/10
20/20 [==============================] - 144s 6s/step - loss: 0.5317 - binary_accuracy: 0.7218 - val_loss: 0.5616 - val_binary_accuracy: 0.8312
Epoch 7/10
20/20 [==============================] - 145s 6s/step - loss: 0.5184 

# EfficientNet-B1

In [24]:
efficientB1_flaire = model_EfficientNetB1('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b1_tf24_imagenet_1000_notop.h5')
history_flair_2 = efficientB1_flaire.fit_generator(
        generator=train_flair,
        steps_per_epoch=len(train_flair),
        epochs=10,
        validation_data=test_flair,
        validation_steps=len(test_flair),
        callbacks=[callback],
        workers=2)

Epoch 1/10
 2/15 [===>..........................] - ETA: 24s - loss: 0.7849 - accuracy: 0.5527  

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


15/15 [==============================] - 176s 9s/step - loss: 0.7240 - accuracy: 0.5975 - val_loss: 0.7085 - val_accuracy: 0.4835
Epoch 2/10
15/15 [==============================] - 106s 7s/step - loss: 0.6503 - accuracy: 0.6272 - val_loss: 0.6651 - val_accuracy: 0.6533
Epoch 3/10
15/15 [==============================] - 114s 7s/step - loss: 0.6167 - accuracy: 0.6558 - val_loss: 0.6481 - val_accuracy: 0.6899
Epoch 4/10
15/15 [==============================] - 115s 7s/step - loss: 0.5875 - accuracy: 0.6845 - val_loss: 0.7009 - val_accuracy: 0.4799
Epoch 5/10
15/15 [==============================] - 104s 6s/step - loss: 0.5753 - accuracy: 0.6924 - val_loss: 0.6812 - val_accuracy: 0.5469
Epoch 6/10
15/15 [==============================] - 103s 6s/step - loss: 0.5648 - accuracy: 0.7085 - val_loss: 0.7128 - val_accuracy: 0.4281


In [25]:
efficientB1_t1w = model_EfficientNetB1('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b1_tf24_imagenet_1000_notop.h5')
history_t1w_2 = efficientB1_t1w.fit_generator(
        generator=train_t1w,
        steps_per_epoch=len(train_t1w),
        epochs=10,
        validation_data=test_t1w,
        validation_steps=len(test_t1w),
        callbacks=[callback],
        workers=2)

Epoch 1/10
16/16 [==============================] - 176s 9s/step - loss: 0.7546 - accuracy: 0.5525 - val_loss: 0.5260 - val_accuracy: 0.9705
Epoch 2/10
16/16 [==============================] - 105s 6s/step - loss: 0.6872 - accuracy: 0.5976 - val_loss: 0.5419 - val_accuracy: 0.9537
Epoch 3/10
16/16 [==============================] - 101s 6s/step - loss: 0.6594 - accuracy: 0.6184 - val_loss: 0.5441 - val_accuracy: 0.9225
Epoch 4/10
16/16 [==============================] - 111s 6s/step - loss: 0.6409 - accuracy: 0.6351 - val_loss: 0.5592 - val_accuracy: 0.9014


In [26]:
efficientB1_t1wce = model_EfficientNetB1('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b1_tf24_imagenet_1000_notop.h5')
history_t1wce_2 = efficientB1_t1wce.fit_generator(
        generator=train_t1wce,
        steps_per_epoch=len(train_t1wce),
        epochs=10,
        validation_data=test_t1wce,
        validation_steps=len(test_t1wce),
        callbacks=[callback],
        workers=2)

Epoch 1/10
19/19 [==============================] - 203s 9s/step - loss: 0.7282 - accuracy: 0.5536 - val_loss: 0.7361 - val_accuracy: 0.2977
Epoch 2/10
19/19 [==============================] - 134s 7s/step - loss: 0.6660 - accuracy: 0.6068 - val_loss: 0.7177 - val_accuracy: 0.3956
Epoch 3/10
19/19 [==============================] - 131s 7s/step - loss: 0.6354 - accuracy: 0.6398 - val_loss: 0.7217 - val_accuracy: 0.4185
Epoch 4/10
19/19 [==============================] - 129s 6s/step - loss: 0.6171 - accuracy: 0.6600 - val_loss: 0.7086 - val_accuracy: 0.4559
Epoch 5/10
19/19 [==============================] - 131s 7s/step - loss: 0.6006 - accuracy: 0.6699 - val_loss: 0.7259 - val_accuracy: 0.4275
Epoch 6/10
19/19 [==============================] - 137s 6s/step - loss: 0.5897 - accuracy: 0.6766 - val_loss: 0.6825 - val_accuracy: 0.5614
Epoch 7/10
19/19 [==============================] - 138s 6s/step - loss: 0.5741 - accuracy: 0.6964 - val_loss: 0.6813 - val_accuracy: 0.5711
Epoch 8/10
19

In [27]:
efficientB1_t2w = model_EfficientNetB1('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b1_tf24_imagenet_1000_notop.h5')
history_t2w_2 = efficientB1_t2w.fit_generator(
        generator=train_t2w,
        steps_per_epoch=len(train_t2w),
        epochs=10,
        validation_data=test_t2w,
        validation_steps=len(test_t2w),
        callbacks=[callback],
        workers=2)

Epoch 1/10
 4/20 [=====>........................] - ETA: 1:33 - loss: 0.7522 - accuracy: 0.5605

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


20/20 [==============================] - 195s 9s/step - loss: 0.7003 - accuracy: 0.5953 - val_loss: 0.6366 - val_accuracy: 0.7601
Epoch 2/10
20/20 [==============================] - 152s 7s/step - loss: 0.6296 - accuracy: 0.6484 - val_loss: 0.5674 - val_accuracy: 0.9241
Epoch 3/10
20/20 [==============================] - 155s 7s/step - loss: 0.6082 - accuracy: 0.6684 - val_loss: 0.6110 - val_accuracy: 0.8182
Epoch 4/10
20/20 [==============================] - 150s 7s/step - loss: 0.5827 - accuracy: 0.6773 - val_loss: 0.5217 - val_accuracy: 0.9440
Epoch 5/10
20/20 [==============================] - 156s 7s/step - loss: 0.5645 - accuracy: 0.7019 - val_loss: 0.5158 - val_accuracy: 0.9296
Epoch 6/10
20/20 [==============================] - 159s 7s/step - loss: 0.5505 - accuracy: 0.7099 - val_loss: 0.5099 - val_accuracy: 0.8982
Epoch 7/10
20/20 [==============================] - 151s 7s/step - loss: 0.5308 - accuracy: 0.7268 - val_loss: 0.5362 - val_accuracy: 0.8804
Epoch 8/10
20/20 [======

# EfficientNet-B2

In [28]:
efficientB2_flaire = model_EfficientNetB2('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b2_tf24_imagenet_1000_notop.h5')
history_flair_3 = efficientB2_flaire.fit_generator(
        generator=train_flair,
        steps_per_epoch=len(train_flair),
        epochs=10,
        validation_data=test_flair,
        validation_steps=len(test_flair),
        callbacks=[callback],
        workers=2)

Epoch 1/10
15/15 [==============================] - 145s 9s/step - loss: 0.8164 - accuracy: 0.5442 - val_loss: 0.6579 - val_accuracy: 0.6515
Epoch 2/10
15/15 [==============================] - 121s 7s/step - loss: 0.6669 - accuracy: 0.6255 - val_loss: 0.6805 - val_accuracy: 0.5630
Epoch 3/10
15/15 [==============================] - 106s 7s/step - loss: 0.6348 - accuracy: 0.6454 - val_loss: 0.5990 - val_accuracy: 0.7480
Epoch 4/10
15/15 [==============================] - 116s 7s/step - loss: 0.6104 - accuracy: 0.6700 - val_loss: 0.5956 - val_accuracy: 0.7176
Epoch 5/10
15/15 [==============================] - 106s 7s/step - loss: 0.5901 - accuracy: 0.6850 - val_loss: 0.5590 - val_accuracy: 0.7793
Epoch 6/10
15/15 [==============================] - 107s 7s/step - loss: 0.5723 - accuracy: 0.6993 - val_loss: 0.5366 - val_accuracy: 0.8061
Epoch 7/10
15/15 [==============================] - 106s 7s/step - loss: 0.5632 - accuracy: 0.7045 - val_loss: 0.5217 - val_accuracy: 0.8427
Epoch 8/10
15

In [29]:
efficientB2_t1w = model_EfficientNetB2('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b2_tf24_imagenet_1000_notop.h5')
history_t1w_3 = efficientB2_t1w.fit_generator(
        generator=train_t1w,
        steps_per_epoch=len(train_t1w),
        epochs=10,
        validation_data=test_t1w,
        validation_steps=len(test_t1w),
        callbacks=[callback],
        workers=2)

Epoch 1/10
16/16 [==============================] - 147s 8s/step - loss: 0.7511 - accuracy: 0.5555 - val_loss: 0.8155 - val_accuracy: 0.1264
Epoch 2/10
16/16 [==============================] - 118s 6s/step - loss: 0.6689 - accuracy: 0.6063 - val_loss: 0.7068 - val_accuracy: 0.4701
Epoch 3/10
16/16 [==============================] - 104s 6s/step - loss: 0.6276 - accuracy: 0.6476 - val_loss: 0.7178 - val_accuracy: 0.4187
Epoch 4/10
16/16 [==============================] - 113s 6s/step - loss: 0.6203 - accuracy: 0.6551 - val_loss: 0.7976 - val_accuracy: 0.2199
Epoch 5/10
16/16 [==============================] - 102s 7s/step - loss: 0.5993 - accuracy: 0.6739 - val_loss: 0.7510 - val_accuracy: 0.3858


In [30]:
efficientB2_t1wce = model_EfficientNetB2('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b2_tf24_imagenet_1000_notop.h5')
history_t1wce_3 = efficientB2_t1wce.fit_generator(
        generator=train_t1wce,
        steps_per_epoch=len(train_t1wce),
        epochs=10,
        validation_data=test_t1wce,
        validation_steps=len(test_t1wce),
        callbacks=[callback],
        workers=2)

Epoch 1/10
19/19 [==============================] - 217s 9s/step - loss: 0.7936 - accuracy: 0.5542 - val_loss: 0.6717 - val_accuracy: 0.5836
Epoch 2/10
19/19 [==============================] - 147s 7s/step - loss: 0.6894 - accuracy: 0.5945 - val_loss: 0.7241 - val_accuracy: 0.3921
Epoch 3/10
19/19 [==============================] - 132s 7s/step - loss: 0.6535 - accuracy: 0.6295 - val_loss: 0.7018 - val_accuracy: 0.4962
Epoch 4/10
19/19 [==============================] - 138s 6s/step - loss: 0.6153 - accuracy: 0.6624 - val_loss: 0.6957 - val_accuracy: 0.5108


In [31]:
efficientB2_t2w = model_EfficientNetB2('../input/efficentnet-b0b5-tensorflow-24-notop/efficientnet-b2_tf24_imagenet_1000_notop.h5')
history_t2w_3 = efficientB2_t2w.fit_generator(
        generator=train_t2w,
        steps_per_epoch=len(train_t2w),
        epochs=10,
        validation_data=test_t2w,
        validation_steps=len(test_t2w),
        callbacks=[callback],
        workers=2)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


Epoch 1/10
20/20 [==============================] - 231s 10s/step - loss: 0.7591 - accuracy: 0.5683 - val_loss: 0.6046 - val_accuracy: 0.9002
Epoch 2/10
20/20 [==============================] - 179s 8s/step - loss: 0.6513 - accuracy: 0.6345 - val_loss: 0.5696 - val_accuracy: 0.9638
Epoch 3/10
20/20 [==============================] - 172s 8s/step - loss: 0.6118 - accuracy: 0.6624 - val_loss: 0.5649 - val_accuracy: 0.9159
Epoch 4/10
20/20 [==============================] - 169s 8s/step - loss: 0.5960 - accuracy: 0.6738 - val_loss: 0.5321 - val_accuracy: 0.8592
Epoch 5/10
20/20 [==============================] - 173s 8s/step - loss: 0.5801 - accuracy: 0.6851 - val_loss: 0.5284 - val_accuracy: 0.9180
Epoch 6/10
20/20 [==============================] - 172s 8s/step - loss: 0.5590 - accuracy: 0.7013 - val_loss: 0.4835 - val_accuracy: 0.9050
Epoch 7/10
20/20 [==============================] - 174s 8s/step - loss: 0.5439 - accuracy: 0.7195 - val_loss: 0.4751 - val_accuracy: 0.9419
Epoch 8/10
2

In [32]:
pred_values = []

# making prediction
def predictions(model, data, dataframe):
    test_pred = model.predict(data, steps=len(data))
    dataframe['pred_y'] = test_pred
    mean_pred = test_pred.mean()
    preds = dataframe.groupby('patient_ids').apply(lambda x: x['pred_y'].max()
        if (x['pred_y'].max() - mean_pred) > (mean_pred - x['pred_y'].min()) 
        else x['pred_y'].min())
    return preds.values

In [33]:
# flaire prediction
flaire1 = predictions(efficientB0_flaire, test_flair, test_df_flair)
flaire2 = predictions(efficientB1_flaire, test_flair, test_df_flair)
flaire3 = predictions(efficientB2_flaire, test_flair, test_df_flair)

# tw1 prediction
t1w1 = predictions(efficientB0_t1w, test_t1w, test_df_t1w)
t1w2 = predictions(efficientB1_t1w, test_t1w, test_df_t1w)
t1w3 = predictions(efficientB2_t1w, test_t1w, test_df_t1w)

# tw1ce prediction
t1wce1 = predictions(efficientB0_t1wce, test_t1wce, test_df_t1wce)
t1wce2 = predictions(efficientB1_t1wce, test_t1wce, test_df_t1wce)
t1wce3 = predictions(efficientB2_t1wce, test_t1wce, test_df_t1wce)

# tw2 predicton
t2w1 = predictions(efficientB0_t2w, test_t2w, test_df_t2w)
t2w2 = predictions(efficientB1_t2w, test_t2w, test_df_t2w)
t2w3 = predictions(efficientB2_t2w, test_t2w, test_df_t2w)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in true_divide


In [34]:
pred_values.append(flaire1)
pred_values.append(flaire2)
pred_values.append(flaire3)
pred_values.append(t1w1)
pred_values.append(t1w2)
pred_values.append(t1w3)
pred_values.append(t1wce1)
pred_values.append(t1wce2)
pred_values.append(t1wce3)
pred_values.append(t2w1)
pred_values.append(t2w2)
pred_values.append(t2w3)

# Submission

In [35]:
all_test_preds = np.array(pred_values)
sub = pd.read_csv(root_dir+'sample_submission.csv') 
sub['MGMT_value'] = all_test_preds.mean(0) 
sub.to_csv("submission.csv", index=False)